In [1]:
from src.data.tushare.info import Description , ChangeName , Calendar
app = Calendar()
app.fetch_and_save()

In [2]:
import pandas as pd

desc = pd.read_feather('./data/DataBase/DB_information_ts/description.feather')
indus = pd.read_feather('./data/DataBase/DB_information_ts/industry.feather')
cname = pd.read_feather('./data/DataBase/DB_information_ts/change_name.feather')
desc

,ts_code,sec_name,industry,exchange_name,list_dt,delist_dt,secid
0,000001.SZ,平安银行,银行,SZSE,19910403,99991231,1
1,000002.SZ,万科A,全国地产,SZSE,19910129,99991231,2
2,000004.SZ,国华网安,软件服务,SZSE,19910114,99991231,4
3,000006.SZ,深振业A,区域地产,SZSE,19920427,99991231,6
4,000007.SZ,全新好,其他商业,SZSE,19920413,99991231,7
...,...,...,...,...,...,...,...
5614,688555.SH,退泽达(退),None,SSE,20200623,20230707,688555
5615,832317.BJ,观典防务(退),None,BSE,20200727,20220426,832317
5616,833874.BJ,泰祥股份(退),None,BSE,20200727,20220718,833874
5617,833994.BJ,翰博高新(退),None,BSE,20200727,20220725,833994


In [3]:
import numpy as np
from typing import Any , Literal
from src.data import load_target_file

class TradeCalendar:
    def __init__(self) -> None:
        cal = pd.read_feather('./data/DataBase/DB_information_ts/calendar.feather').loc[:,['calendar' , 'trade']]

        trd = cal[cal['trade'] == 1].reset_index(drop=True)
        trd['td'] = trd['calendar']
        trd['pre'] = trd['calendar'].shift(1, fill_value=-1)
        trd = cal.merge(trd.drop(columns='trade') , on = 'calendar' , how = 'left').ffill()
        trd['cd_index'] = np.arange(len(trd))
        trd['td_index'] = trd['trade'].cumsum()
        trd = trd.astype(int)

        self.calendar = trd.set_index('calendar')
        self.cal_cal = trd.set_index('cd_index')
        self.cal_trd = trd[trd['trade'] == 1].set_index('td_index')
    
    def __td_transform(self , td , as_numpy : bool) -> Any:
        if isinstance(td , pd.Series): td = td.to_numpy()
        if as_numpy and isinstance(td , int): td = np.array([td])
        return td 

    def td(self , date , as_numpy = True): 
        td = self.calendar.loc[date , 'td']
        return self.__td_transform(td , as_numpy)
    
    def pre(self , date , as_numpy = True):
        td = self.calendar.loc[date , 'pre']
        return self.__td_transform(td , as_numpy)
    
    def offset(self , date , n : Any = 0 , type : Literal['t' , 'c'] = 't' , as_numpy = True):
        if type == 't':
            d_index = self.calendar.loc[date , 'td_index'] + n
            d_index = np.maximum(np.minimum(d_index , len(self.cal_trd) - 1) , 0)
            td = self.cal_trd.loc[d_index , 'calendar']
        else:
            d_index = self.calendar.loc[date , 'cd_index'] + n
            d_index = np.maximum(np.minimum(d_index , len(self.cal_cal) - 1) , 0)
            td = self.cal_cal.loc[d_index , 'calendar']
        return self.__td_transform(td , as_numpy)
    
    @property
    def calendar_start(self): return self.calendar.index.min()
    @property
    def calendar_end(self): return self.calendar.index.max()
        
calendar = TradeCalendar()
calendar.calendar_start

19901219

In [21]:


date = 20120105

def risk_universe(date : int):
    list_days = 252
    redempt_tmv_pct = 0.8

    new_desc = desc[(desc['list_dt'] <= date) & (desc['delist_dt'] > date)].copy()
    new_desc['list_dt'] = np.maximum(new_desc['list_dt'] , calendar.calendar_start)
    new_desc = new_desc.set_index('secid')

    val = load_target_file('trade_ts' , 'day_val' , date)
    val['weight'] = val['circ_mv'] / 1e8
    val = val.set_index('secid').reindex(new_desc.index)

    cal_after = calendar.offset(new_desc['list_dt'] , list_days , 't')

    trd = load_target_file('trade_ts' , 'day' , calendar.offset(date , -21 , 't')).loc[:,['secid','status']]
    trd = load_target_file('trade_ts' , 'day' , date).loc[:,['secid','status']].merge(trd , on = 'secid' , how = 'left').\
        set_index('secid').reindex(new_desc.index).fillna(0)
    
    new_cname = cname[cname['secid'] >= 0].sort_values(['secid','ann_date','start_date'])
    new_cname = new_cname[new_cname['start_date'] <= date].drop_duplicates('secid' , keep = 'last')
    new_cname = new_cname[new_cname['change_reason'].isin(['终止上市', '暂停上市' , 'ST', '*ST', ])]
    
    # trade status are 1.0 this day or 1 month ealier
    rule0 = ((trd['status_x'] == 1) | (trd['status_y'] == 1))

    # list date 1 year eailier and not delisted or total mv in the top 90%
    rule1 = ((new_desc['delist_dt'] > date) & (cal_after <= date)) | (val['total_mv'].rank(pct = True , na_option='bottom') >= redempt_tmv_pct)

    # not st
    rule2 = ~new_desc.index.isin(new_cname['secid'])


    new_desc['estuniv'] = 1.0 * (rule0 & rule1 & rule2)
    new_desc['weight'] = val['weight'].fillna(0).values

    return new_desc.reset_index(drop=True)
df = risk_universe(date)
df.iloc[:10]

secid
1     477.914700
2     700.246929
4       6.148198
6      38.810776
7      12.133728
8       4.676973
9     102.978356
10           NaN
11      7.925437
12    105.766239
Name: weight, dtype: float64
secid
1     477.914700
2     700.246929
4       6.148198
6      38.810776
7      12.133728
8       4.676973
9     102.978356
10      0.000000
11      7.925437
12    105.766239
Name: weight, dtype: float64


,ts_code,sec_name,industry,exchange_name,list_dt,delist_dt,estuniv,weight
0,000001.SZ,平安银行,银行,SZSE,19910403,99991231,1.0,477.914700
1,000002.SZ,万科A,全国地产,SZSE,19910129,99991231,1.0,700.246929
2,000004.SZ,国华网安,软件服务,SZSE,19910114,99991231,1.0,6.148198
3,000006.SZ,深振业A,区域地产,SZSE,19920427,99991231,1.0,38.810776
4,000007.SZ,全新好,其他商业,SZSE,19920413,99991231,1.0,12.133728
5,000008.SZ,神州高铁,运输设备,SZSE,19920507,99991231,1.0,4.676973
6,000009.SZ,中国宝安,电气设备,SZSE,19910625,99991231,1.0,102.978356
7,000010.SZ,美丽生态,建筑工程,SZSE,19951027,99991231,0.0,0.000000
8,000011.SZ,深物业A,房产服务,SZSE,19920330,99991231,1.0,7.925437
9,000012.SZ,南玻A,玻璃,SZSE,19920228,99991231,1.0,105.766239


In [ ]:
from typing import Any
from src.data import load_target_file
def risk_weight(date , secid : Any = None):
    val = load_target_file('trade_ts' , 'day_val' , date).loc[:,['secid','circ_mv']]
    val['weight'] = val['circ_mv'] / 1e8
    val = val.drop(columns='circ_mv')
    if secid is not None: val = val.set_index('secid').loc[secid].reset_index()
    return val
risk_weight(20120105).iloc[:10]

In [ ]:
rsk = pd.read_feather('./data/DataBase/DB_models/risk_exp/2012/risk_exp.20120105.feather')
rsk.iloc[:10]

In [ ]:
val = pd.read_feather('./data/DataBase/DB_trade_ts/day_val/2012/day_val.20120105.feather')

In [ ]:
val

In [ ]:
val['total_mv'].rank(pct = True , na_option='bottom')

In [ ]:
# ['终止上市', 'ST', '*ST', '暂停上市']
date = 20240105
cname[(cname['change_reason'].isin(['终止上市', '暂停上市' , 'ST', '*ST', ]) ) & 
      (cname['entry_dt'] <= date) & (cname['remove_dt'] >= date)]['name'].unique()

In [ ]:
info['industry'].unique()

In [ ]:
indus['l1_name'].unique().shape

In [ ]:
indus['l2_name'].unique()